In [1]:
import SIRD_Model

import numpy as np
import csv
import matplotlib.pyplot as plt
import platform

In [2]:
if platform.system() == "Windows":
    pathc="..\\Data\\Italian Data\\"
elif platform.system() == "Linux":
    pathc="../Data/Italian Data/"
    
filename = "National Data.csv"

dates, infectRaw, recovRaw, deadRaw = SIRD_Model.loadData(pathc + filename)

In [36]:
import scipy.optimize as opt

#note paramArg is in the format of [q, beta, gamma, nu]
def errorSIRD(paramArg, pop, I, R, D, lamda, w): #the custom error function for SIRD    
    
    q = paramArg[0]
    b = paramArg[1:5]
    params = paramArg[4:] #beta3, gamma, nu
    params[0] = b[1] #beta0, gamma, nu
    
    #note: beta = b0 / (1 + (I*b1 + D*b2)^b3)
    #use b0 in the place of b0, and multiply first column by 1/(1+(I*b1+D*b2)^b3)
    
    S = q*pop - I - R - D #q*Pop = S + I + R + D
    
    y, A = SIRD_Model.getSIRDMatrices(S, I, R, D)
    
    #transform location where beta is needed and multiply by 1/(1+(I*b1+D*b2)^b3)
    #A[:,0,0] = A[:,0,0] / (1 + (I[0:-1]*b[1] + D[0:-1]*b[2])**b[3])
    #A[:,1,0] = A[:,1,0] / (1 + (I[0:-1]*b[1] + D[0:-1]*b[2])**b[3])
    
    totalError = 0
    #see paper for optimization function
    T = len(A)
    for t in range(T):
        totalError = totalError + (w**(T - t))*(np.linalg.norm((A[t] @ params) - y[t].transpose(), ord=2)**2)
    
    #return (1.0/T) * np.linalg.norm((A @ params) - y.transpose(), ord=2)**2  + lamda*np.linalg.norm(params, ord=1)
    totalError = (1.0/T)*totalError #divide by timeframe
    totalError = totalError + lamda*np.linalg.norm(params, ord=1) #regularization error
    return totalError



In [39]:
pop = 60000000
numDays = 30 #len(infectRaw) #max days
infect = infectRaw[0:numDays]
recov = recovRaw[0:numDays]
dead = deadRaw[0:numDays]

boundQ = (0,1) #q should be between 0 and 1

#Beta = b0/(1+(I*b1 + D*b2)^b3)
boundB0 = (0,10)
boundB1 = (0,10)
boundB2 = (0,10)
boundB3 = (0,5)

boundG = (0,1) #recovery rate
boundN = (0,1) #death rate

lamda = 10 #regularization weight
wVal = .9 #weight decay

constraints = [boundQ, boundB0, boundB1, boundB2, boundB3, boundG, boundN]
constArg = (pop, infect, recov, dead, lamda, wVal)
initGuess = [.1, .1,.01,.01,1, .1, .01] #init values, q, b0,b1,b2,b3, gamma, nu

#result = opt.shgo(errorSIRD, constraints, args=constArg)
result = opt.fmin(errorSIRD, initGuess, args=constArg)

print(result)

[-1.42512091e+06  2.02440779e+06  1.34431254e-01  1.15766511e+05
  1.34431254e-01  1.83253110e-02  1.57551855e-02]


In [44]:
def getLinVarsSIRD(paramArg, pop, I, R, D, lamda, w): #calculate the linear vars for the SIRD model, b0, gamma, nu  
    
    q = paramArg[0]
    b1 = paramArg[1]
    b2 = paramArg[2]
    b3 = paramArg[3]
    
    #b0, gamma, nu should be solved for
    
    #note: beta = b0 / (1 + (I*b1 + D*b2)^b3)
    
    S = q*pop - I - R - D #q*Pop = S + I + R + D
    
    nextIterMatrix, sirdMatrix = getSIRDMatricesFlat(suscept, infect, recov, dead)
    #construct y and A, see paper for solving the lasso optimization
    T = int(len(nextIterMatrix)/4)
    y = np.zeros((T*4, 1))
    A = np.zeros((T*4, 3))
    for t in range(T):
        y[4*t+0] = nextIterMatrix[4*t+0] * np.sqrt(w**(T - t))
        y[4*t+1] = nextIterMatrix[4*t+1] * np.sqrt(w**(T - t))
        y[4*t+2] = nextIterMatrix[4*t+2] * np.sqrt(w**(T - t))
        y[4*t+3] = nextIterMatrix[4*t+3] * np.sqrt(w**(T - t))

        #transform location where beta is needed and multiply by 1/(1+(I*b1+D*b2)^b3)
        sirdMatrix[4*t+0] = sirdMatrix[4*t+0] / (1 + (I[t]*b1 + D[t]*b2)**b3)
        sirdMatrix[4*t+0] = sirdMatrix[4*t+1] / (1 + (I[t]*b1 + D[t]*b2)**b3)
        
        A[4*t+0] = sirdMatrix[4*t+0] * np.sqrt(w**(T - t))
        A[4*t+1] = sirdMatrix[4*t+1] * np.sqrt(w**(T - t))
        A[4*t+2] = sirdMatrix[4*t+2] * np.sqrt(w**(T - t))
        A[4*t+3] = sirdMatrix[4*t+3] * np.sqrt(w**(T - t))
            
            
    model = linear_model.Lasso(alpha=lamda, fit_intercept=False, positive=True)
    model.fit(A,y)
    params = model.coef_
    
    totalError = (1.0/T) * np.linalg.norm((A @ params) - y.transpose(), ord=2)**2  + lamda*np.linalg.norm(params, ord=1)
    return params

def gridNonLinVars(paramArg, constraints, varResols, pop, I, R, D, lamda, w): #solve for non linear vars, q, b1, b2, b3
    #note: paramArg = [b0, gamma, nu]
    #b0 = paramArg[0]
    #gamma = paramArg[1]
    #nu = paramArg[2]
    
    varSteps = constraints[:,0] + (constraints[:,1] - constraints[:,0])/varResols[:] #min + (max - min)/resolution
    
    #note beta = b0/(1 + (b1*I + b2*D)^b3)
    #assume starting vals as best starting value
    minVars = constraints[:,0]
    minCost = errorSIRD(minVars + paramArg, pop, I, R, D, lamda, w) #the custom error function for SIRD
        
    currVars = minVars
    currCost = minCost
    varIndex = 0 #which var to iterate
    #while the var isn't above it's max
    while(varIndex != len(currVars)): #while varIndex is in range, ends when everything is iterated
        currVars[varIndex] = currVars[varIndex] + varSteps[varIndex] #iterate variable
        
        print(currVars)
        
        if(currVars[varIndex] > constraints[varIndex,1]): #var value is out of range
            currVars[varIndex] = constraints[varIndex,0] #reset to minimum
            varIndex = varIndex + 1 #move to iterating the next variable
        else: #current variable is in range, go back to iterating first variable
            varIndex = 0

In [46]:
pop = 60000000
numDays = 30 #len(infectRaw) #max days
infect = infectRaw[0:numDays]
recov = recovRaw[0:numDays]
dead = deadRaw[0:numDays]

boundQ = (0,1) #q should be between 0 and 1

#Beta = b0/(1+(I*b1 + D*b2)^b3)
boundB0 = (0,10)
boundB1 = (0,10)
boundB2 = (0,10)
boundB3 = (0,5)

boundG = (0,1) #recovery rate
boundN = (0,1) #death rate

lamda = 10 #regularization weight
wVal = .9 #weight decay

constraints = [boundQ, boundB0, boundB1, boundB2, boundB3, boundG, boundN]
constArg = (pop, infect, recov, dead, lamda, wVal)
initGuess = [.1, .1,.01,.01,1, .1, .01] #init values, q, b0,b1,b2,b3, gamma, nu

gridNonLinVars([.1,.1,.1], [boundQ, boundB1, boundB2, boundB3], [10,10,10,10], *constArg)

TypeError: list indices must be integers or slices, not tuple